In [9]:
import json
import nltk
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from nltk.corpus import wordnet

In [2]:
#load the corpus as a string

source_text = ""

with open("/Users/jxu2/Documents/Work/6 ITP/sem 3/a2z/word2vec/hm_only.txt", "r") as f:
    for line in f:
        source_text += line
        source_text += " "

In [7]:
#split string into sentences

tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
raw_sentences = tokenizer.tokenize(source_text)

I went on a successful date with someone I felt sympathy and connection with.


In [11]:
#clean each sentence, saving semantically meaningful words into an array

def sentence_to_wordlist(raw):
    words = raw.lower().split()

    stopWords = set(stopwords.words('english'))

    tokenized = tokenizer.tokenize(raw)
    real_words = raw.lower().split()

    sentenceArray = []

    for word in real_words:
        if word in stopWords:
            continue
        elif wordnet.synsets(word):
            sentenceArray.append(word)
            
#     # Create a list of words
#     sentenceArray = [word for word in real_words if word not in stopWords]
    return sentenceArray

In [12]:
cleaned_sentences = []
for raw in raw_sentences:
    cleaned_sentences.append(sentence_to_wordlist(raw))

In [16]:
print(cleaned_sentences[0])

['went', 'successful', 'date', 'someone', 'felt', 'sympathy', 'connection']


In [18]:
#build the w2v model

model = w2v.Word2Vec(sg=1,seed=1,workers= 4,size=100,min_count=3,window=15,sample=1e-3)
model.build_vocab(cleaned_sentences)
model.train(cleaned_sentences, total_examples=model.corpus_count,epochs=model.iter)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(3177930, 3667610)

In [19]:
print(model)

Word2Vec(vocab=9739, size=100, alpha=0.025)


In [20]:
#save the model as a lookup dictionary json

vectorsDict = {}
for sentence in cleaned_sentences:
    for word in sentence:
        keys = list(vectorsDict.keys())
        if word not in keys:
            try:
                vectorsDict[word] = model.wv[word].tolist()
            except:
                continue

In [21]:
with open("happyDB_vectors.json", "w+") as f:
    json.dump(vectorsDict, f)